In [1]:
import tensorflow as tf

/Users/Linh-Main/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions f

In [3]:
data = tf.placeholder(tf.float32, [None, 784])
labels = tf.placeholder(tf.float32, [None, 10])

sizes = [300,10]
input_layer = tf.reshape(data, [-1, 28, 28, 1])

with tf.variable_scope('conv1') as cope:
    kernels = tf.get_variable('kernel1',[5,5,1,16],
                              initializer = tf.truncated_normal_initializer())
    biases = tf.get_variable('biases2', [16],dtype =tf.float32,
                            initializer = tf.zeros_initializer())
    conv = tf.nn.conv2d(input = input_layer,
                            filter = kernels,
                            strides=[1,1,1,1],
                           padding = 'SAME')
    conv1 = tf.nn.relu(conv+biases)
    
with tf.variable_scope('pool1') as scope:    
    pool1 = tf.nn.max_pool(value = conv1, 
                           ksize=[1,2,2,1], 
                           strides=[1,2,2,1],
                          padding = 'SAME')
    
with tf.variable_scope('conv2') as cope:
    kernels = tf.get_variable('kernel2',[5,5,16,32],
                              initializer = tf.truncated_normal_initializer())
    biases = tf.get_variable('biases2',[32],dtype =tf.float32,
                            initializer = tf.zeros_initializer())
    conv = tf.nn.conv2d(input =pool1,filter = kernels,
                         strides = [1,1,1,1],
                        padding = 'SAME')
    conv2 = tf.nn.relu(conv+biases)

with tf.variable_scope('pool2') as scope:
    pool2 = tf.nn.max_pool(value =conv2, 
                        ksize=[1,2,2,1], 
                        strides=[1,2,2,1],
                          padding = 'SAME')
    
with tf.variable_scope('dense1') as cope:
    weights = tf.get_variable('weight1',[7*7*32,sizes[0]],
                                   initializer = tf.truncated_normal_initializer(stddev = (7*7*32)**(-0.5)))
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 32])
    pool_BN = tf.layers.batch_normalization(inputs = pool2_flat,
                                              center=True, 
                                              scale=True, 
                                              training=True)
    dense = tf.nn.relu(tf.matmul(pool_BN,weights))

    dropout = tf.nn.dropout(dense, keep_prob=0.6)
    
with tf.variable_scope('logits') as scope:
    weights = tf.get_variable('weight1',[sizes[0],sizes[1]],
                                   initializer = tf.truncated_normal_initializer(stddev = sizes[0]**(-0.5)))
    dropout_BN = tf.layers.batch_normalization(dropout,
                          center = True,
                          scale = True,
                          training = True)
    logits = tf.matmul(dropout_BN,weights)
    
    dense_BN = tf.layers.batch_normalization(dense,
                                center = True,
                                scale = True,
                                training = True)
    
    logits_wo_drop = tf.matmul(dense_BN,weights)

cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels = labels, 
                                                logits = logits) 

# define evaluation metrics
correct_prediction = tf.equal(tf.argmax(logits_wo_drop,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [4]:
def train():
    import sys
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    optimize = optimizer.minimize(cross_entropy)
    
    epochs = 10
    batch_size = 60
    n_batches = int(len(mnist.train.images)/batch_size)
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()

    # create saver
    saver = tf.train.Saver()

    predictions_o = []
    for epoch in range(epochs):
        avg_cost =0
        for i in range(n_batches):
            inputs_batch, labels_batch = mnist.train.next_batch(batch_size)
            _,cost = sess.run([optimize,cross_entropy],feed_dict={data: inputs_batch, labels: labels_batch})
            avg_cost += cost/n_batches
            if i%100 == 0:
                sys.stdout.write('\r batch_cost {}'.format(cost))
            
        test_acc = sess.run(accuracy, feed_dict={data: mnist.test.images, labels:mnist.test.labels})
        print("\n Epoch:", (epoch + 1), "cost =", "{:.4f}".format(avg_cost), "test accuracy: {:.4f}".format(test_acc))
            #save_path = saver.save(sess, "C:/Users/Daniel1/Desktop/ML Competition/model_checkpoints/{}rz_mb_60_conv0864_200_100_50".format(epoch))

    print("\n Running stats:")
    sess.close()
    



In [5]:
%%time
train()

 batch_cost 0.127510339021682745
 Epoch: 1 cost = 0.1383 test accuracy: 0.9865
 batch_cost 0.050903130322694787
 Epoch: 2 cost = 0.0715 test accuracy: 0.9864
 batch_cost 0.0131026888266205796
 Epoch: 3 cost = 0.0598 test accuracy: 0.9881
 batch_cost 0.029270796105265617
 Epoch: 4 cost = 0.0552 test accuracy: 0.9896
 batch_cost 0.075201980769634253
 Epoch: 5 cost = 0.0486 test accuracy: 0.9901
 batch_cost 0.05920065939426422766
 Epoch: 6 cost = 0.0458 test accuracy: 0.9906
 batch_cost 0.029319405555725098
 Epoch: 7 cost = 0.0444 test accuracy: 0.9901
 batch_cost 0.118512488901615148
 Epoch: 8 cost = 0.0432 test accuracy: 0.9906
 batch_cost 0.0060415747575461864
 Epoch: 9 cost = 0.0403 test accuracy: 0.9898
 batch_cost 5.435773346107453e-054
 Epoch: 10 cost = 0.0399 test accuracy: 0.9898

 Running stats:
CPU times: user 33min 36s, sys: 5min 30s, total: 39min 6s
Wall time: 19min 47s
